In [59]:
# Storage Account Parameters
storageAccountName      = ""
storageAccountContainer = ""
storageAccountDirectory = "" # do not include a leading slash
storageLinkedServiceName = ""

# Event Hub Paramaters
keyVaultName = ""
linkedServiceName = ""
secretName = ""
ehName = ''

In [60]:
storageAccountFullPath  = f'abfss://{storageAccountContainer}@{storageAccountName}.dfs.core.windows.net/{storageAccountDirectory}'

mssparkutils.fs.mount(storageAccountFullPath,"/faredata_raw",{"linkedService":storageLinkedServiceName}) 

jobId = mssparkutils.env.getJobId()


In [61]:
mssparkutils.fs.ls(f"synfs:/{jobId}/faredata_raw")

In [62]:
pip install azure-eventhub

In [63]:
import asyncio
from multiprocessing.connection import wait
from multiprocessing.sharedctypes import Value
from os import environ
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
import csv
from zipfile import ZipFile
from pathlib import Path
import io
import time
import sys
import os
import nest_asyncio

dir = f'/synfs/{jobId}/faredata_raw'
ehConnectionString = mssparkutils.credentials.getSecret(keyVaultName,secretName,linkedServiceName)


In [64]:
def getCSVFiles():
    csvCount=0
    listCSVs = []

    for child in Path(dir).glob('*.csv'):
        listCSVs.append(child)
        csvCount += 1
    
    return csvCount,listCSVs

# Function to send to Event Hubs
async def main(listOfLines):
    # Create a producer client to send messages to the event hub.
    # Specify a connection string to your event hubs namespace and
    # the event hub name.
    producer = EventHubProducerClient.from_connection_string(conn_str=ehConnectionString, eventhub_name=ehName)
    async with producer:
        # Create a batch.
        event_data_batch = await producer.create_batch()

        # Add events to the batch.
        for line in listOfLines:
            event_data_batch.add(EventData(line))
        
        # Send the batch of events to the event hub.
        await producer.send_batch(event_data_batch)

In [65]:
ctCsvFiles, csvFiles = getCSVFiles()
for csvFile in csvFiles:
    openFile = io.open(csvFile,'r')
    count = 0
    print(f"Starting {csvFile}")
    lineList = []

    while True:
        count +=1

        line = openFile.readline().strip()

        # I guarantee there's a more elegant way to do this. However, I want to control the number of lines going through and how they are shaped.
        if line.startswith("medallion") == False and line != "":
            splitLine       = line.split(",")
            buildJsonString = "{"
            buildJsonString = f"{buildJsonString}\"medallion\":\"{splitLine[0]}\","
            buildJsonString = f"{buildJsonString}\"hack_license\":\"{splitLine[1]}\","
            buildJsonString = f"{buildJsonString}\"vendor_id\":\"{splitLine[2]}\","
            buildJsonString = f"{buildJsonString}\"pickup_datetime\":\"{splitLine[3]}\","
            buildJsonString = f"{buildJsonString}\"payment_type\":\"{splitLine[4]}\","
            buildJsonString = f"{buildJsonString}\"fare_amount\":\"{splitLine[5]}\","
            buildJsonString = f"{buildJsonString}\"surcharge\":\"{splitLine[6]}\","
            buildJsonString = f"{buildJsonString}\"mta_tax\":\"{splitLine[7]}\","
            buildJsonString = f"{buildJsonString}\"tip_amount\":\"{splitLine[8]}\","
            buildJsonString = f"{buildJsonString}\"tolls_amount\":\"{splitLine[9]}\","
            buildJsonString = f"{buildJsonString}\"total_amount\":\"{splitLine[10]}\""
            buildJsonString = f"{buildJsonString}}}"
            lineList.append(buildJsonString)        

        if count % 3500 == 0:
            try:
                await(main(lineList))
            except Exception as e:
                print(f"Batch {count} in file {csvFile} failed to send to Event Hubs")
                print(e)
            finally:                
                lineList = []

        if count % 12000 == 0:
            print(f"{count} lines sent")
            sys.stdout.flush()

        if not line:
            break
    
    # Send remaining lines to event hubs since not all files will show a remainder of 0 when dividing by 3500
    if count % 3500 != 0:
        try:
            await(main(lineList))
        except Exception as e:
            print(f"Batch {count} in file {csvFile} failed to send to Event Hubs")
            print(e)
        finally:                
            lineList = []
    
    openFile.close()

In [66]:
mssparkutils.fs.unmount("/faredata_raw") 